In [0]:
import pandas as pd
import numpy as np
import os
from skimage.transform import resize as imresize
from tqdm import tqdm

import imageio


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
%cd /gdrive/My\ Drive/

/gdrive/My Drive


In [4]:
# from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from skimage.transform import resize as imresize

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split

In [0]:

BATCH_SIZE = 16
EPOCHS = 30
RANDOM_STATE = 11

CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

# Dense layers set
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

# Conv. layers set
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

# simple model 
def get_model():
    inp_img = Input(shape=(51, 51, 3))

    # 51
    conv1 = conv_layer(inp_img, 64, zp_flag=False)
    conv2 = conv_layer(conv1, 64, zp_flag=False)
    mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # 23
    conv3 = conv_layer(mp1, 128, zp_flag=False)
    conv4 = conv_layer(conv3, 128, zp_flag=False)
    mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # 9
    conv7 = conv_layer(mp2, 256, zp_flag=False)
    conv8 = conv_layer(conv7, 256, zp_flag=False)
    conv9 = conv_layer(conv8, 256, zp_flag=False)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # 1
    # dense layers
    flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)
    
    # The first 50 epochs are used by Adam opt.
    # Then 30 epochs are used by SGD opt.
    
    #mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    mypotim = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model


def get_callbacks(filepath, patience=5):
    lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=patience, verbose=1)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [lr_reduce, msave]

def train_model(gmodel, img, target):
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE
                                                        )
    gen = ImageDataGenerator(
            rotation_range=360.,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )
    gmodel.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
               steps_per_epoch=10*len(x_train)/BATCH_SIZE,
               epochs=EPOCHS,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid))
    




In [0]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('/gdrive/My Drive/plant-seedlings'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

In [0]:
def img_reshape(img):
    img = imresize(img, (51, 51, 3))
    return img

# get image tag
def img_label(path):
    return str(str(path.split('/')[-1]))

# get plant class on image
def img_class(path):
    return str(path.split('/')[-2])

# fill train and test dict
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = imageio.imread(p)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            some_dict['class'].append(img_class(p))

    return some_dict


In [0]:
from keras.utils import to_categorical

In [10]:
train_dict, test_dict = reader()
X_train = np.array(train_dict['image'])
y_train = to_categorical(np.array([CLASS[l] for l in train_dict['class']]))

X_test = np.array(test_dict['image'])
label = test_dict['label']



Root:
/gdrive/My Drive/plant-seedlings
Dirs:
['test', 'train']
Root:
/gdrive/My Drive/plant-seedlings/train
Dirs:
['Scentless Mayweed', 'Maize', 'Small-flowered Cranesbill', 'Shepherds Purse', 'Black-grass', 'Loose Silky-bent', 'Charlock', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Cleavers', 'Sugar beet']


Start fill test_dict: 100%|########################| 794/794 [04:34<00:00,  3.04it/s]


In [22]:
gmodel = get_model()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 51, 51, 3)         0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 49, 49, 64)        1792      
_________________________________________________________________
batch_normalization_64 (Batc (None, 49, 49, 64)        256       
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 47, 47, 64)        36928     
_________________________________________________________________
batch_normalization_65 (Batc (None, 47, 47, 64)        256       
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 47, 47, 64)        0   

In [23]:
gmodel

In [26]:
train_model(gmodel, X_train, y_train)


Epoch 1/30
2375/2375 [==============================] - 135s 57ms/step - loss: 1.3209 - acc: 0.5482 - val_loss: 1.6171 - val_acc: 0.4779
Epoch 2/30
2375/2375 [==============================] - 106s 45ms/step - loss: 0.8031 - acc: 0.7261 - val_loss: 0.6587 - val_acc: 0.7800
Epoch 3/30
2375/2375 [==============================] - 105s 44ms/step - loss: 0.6482 - acc: 0.7775 - val_loss: 0.8175 - val_acc: 0.7053
Epoch 4/30
2375/2375 [==============================] - 105s 44ms/step - loss: 0.5317 - acc: 0.8139 - val_loss: 0.8530 - val_acc: 0.7179
Epoch 5/30
2375/2375 [==============================] - 106s 45ms/step - loss: 0.4758 - acc: 0.8360 - val_loss: 0.3976 - val_acc: 0.8863
Epoch 6/30
2375/2375 [==============================] - 106s 45ms/step - loss: 0.4233 - acc: 0.8519 - val_loss: 0.4689 - val_acc: 0.8484
Epoch 7/30
2375/2375 [==============================] - 106s 45ms/step - loss: 0.3846 - acc: 0.8645 - val_loss: 0.3420 - val_acc: 0.8905
Epoch 8/30
2375/2375 [===================

In [0]:
def test_model(gmodel, img, label):
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    sub = pd.DataFrame({"file": label,
                         "species": [INV_CLASS[p] for p in pred]})
    print(sub)

In [39]:
test_model(gmodel, X_test, label)

794/794 [==============================] - 1s 698us/step
              file                    species
0    976e4e079.png           Common Chickweed
1    93079d970.png                 Sugar beet
2    856f2910a.png  Small-flowered Cranesbill
3    1d0cbd819.png           Loose Silky-bent
4    4bbfd1e05.png                   Cleavers
5    c0461776c.png           Common Chickweed
6    b31292706.png           Common Chickweed
7    29bab7cad.png           Common Chickweed
8    37297a64c.png           Common Chickweed
9    f2dc546ca.png                   Cleavers
10   b6a3f7876.png                Black-grass
11   8874bba69.png           Loose Silky-bent
12   1459e96a0.png                   Cleavers
13   9a3f20121.png           Loose Silky-bent
14   5315c2dca.png           Loose Silky-bent
15   b7a7f6390.png                    Fat Hen
16   20e562fd5.png                 Sugar beet
17   e3f50adfc.png                   Cleavers
18   6ba4ef411.png                   Cleavers
19   acdb75e00.png     

In [31]:
gmodel.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 51, 51, 3)         0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 49, 49, 64)        1792      
_________________________________________________________________
batch_normalization_64 (Batc (None, 49, 49, 64)        256       
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 47, 47, 64)        36928     
_________________________________________________________________
batch_normalization_65 (Batc (None, 47, 47, 64)        256       
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 47, 47, 64)        0   

In [0]:
# TRain accuracy is 94%
# Test accuracy is 93%